In [ ]:
# default_exp measures

# Measures

> Compute behavioural measures from raw transaction data

## Overview

A behavioural measure is a number derived from a player's transaction data. This data needs to have specific columns depending on the measure you want to calculate. For example, measures in the bet domain require the **bet_size** column. For each of the measures you plan to use, make sure that the data you have in the gamba standard format (see `check_data`) has the correct values.

Once your data has all of the right columns, the simplest way to get started is to call the `create_measures_table` method below, giving it one or more players bets in the gamba standard format.

In [ ]:
import pandas as pd
#export
def create_measures_table(all_player_bets, measures, daily=False):
    "Creates a measures table from a collection of bets in the gamba standard format. The second parameter 'measures' should be a list of names from the measures module. See `available_measures` for currently available options."
    
    # get unique players
    unique_players = list(set(all_player_bets['player_id'].values))
    
    # create holding structure for each of the measures
    measures_table_values = [unique_players]
    

    # for each of the measures
    for measure in measures:
        all_players_measures = []
        
        # for each of the players
        for player in unique_players:
            
            player_bets = all_player_bets[all_player_bets['player_id'] == player]
            # compute and add to holding structure
            this_measure = eval(measure + '(player_bets)')
            
            all_players_measures.append(this_measure)
            
        measures_table_values.append(all_players_measures)
    
    # turn into tidy dataframe
    measures_table = pd.DataFrame()
    measures_table['player_id'] = unique_players
    for x, measure in enumerate(measures):
        measures_table[measure] = measures_table_values[x + 1]
    
    return measures_table

> Note: The `create_measures_table` method is perhaps the most useful thing in the entire library.

## Measure Domains

Not all measures need the same data to be computed. The gamba library distinguishes between three primary domains (time, cost, and loss), and includes a final *other* domain to include everything else.

### Time Domain

The time domain consists of behavioural measures that can be computed using only the knowledge of when the bets took place. This means using only values from the ***bet_time*** column in the player bet dataframe.

In [ ]:
#export
def duration(player_bets):
    "The number of days between the first bet and the last."
    check_measure_data(player_bets, ["bet_time"])
    player_bets.sort_values("bet_time", inplace=True)
    player_bets.reset_index(drop=True, inplace=True)

    first_day = player_bets.iloc[0]["bet_time"]
    last_day = player_bets.iloc[len(player_bets) - 1]["bet_time"]
    # add one to make it interpret as 'days where betting has occurred'
    age_in_days = (last_day.date() - first_day.date()).days + 1
    return age_in_days

In [ ]:
#export
def frequency(player_bets):
    "The percentage of days within the `duration` that included at least one bet. Note that this method returns the actual percentage value e.g. 75, **not** the raw number of days or values in the interval [0,1]."
    check_measure_data(player_bets, ["bet_time"])
    player_bets.sort_values("bet_time", inplace=True)
    player_bets.reset_index(drop=True, inplace=True)
    age_in_days = duration(player_bets)

    player_bets["bet_date"] = player_bets["bet_time"].dt.date
    first_day = player_bets.iloc[0]["bet_time"]

    betting_days = 0
    for day in range(age_in_days):
        current_date = (first_day + datetime.timedelta(days=day)).date()
        bets_today = player_bets.iloc[player_bets["bet_date"].values == current_date]
        if len(bets_today) != 0:
            betting_days += 1

    frequency_percentage = (betting_days / age_in_days) * 100
    return frequency_percentage

In [ ]:
#export
def number_of_bets(player_bets, daily=False):
    "The total number of bets made."

    if daily:        
        check_measure_data(player_bets, ["bet_count"])
        return player_bets["bet_count"].sum()
    
    return len(player_bets)

In [ ]:
#export
def mean_bets_per_day(player_bets, daily=False):
    "The average (mean) number of bets made on days where betting took place."
    
    if daily:
        
        age_in_days = duration(player_bets)
        frequency_percentage = frequency(player_bets)

        betting_days = (frequency_percentage / 100) * age_in_days

        mean_bets_per_day = (
            number_of_bets(player_bets, daily=True) / betting_days  # taken from LaBrie 2008
        )
        return mean_bets_per_day
    
    # otherwise compute regular mean bets per day measure
    age_in_days = duration(player_bets)
    frequency_percentage = frequency(player_bets)

    betting_days = (frequency_percentage / 100) * age_in_days

    mean_bets_per_day = len(player_bets) / betting_days  # taken from LaBrie 2008
    return mean_bets_per_day

In [ ]:
#export
def intensity(player_bets):
    "Mean number of bets per active betting day in first month, if data set contains daily aggregate data."

    first_day = player_bets.iloc[0]["bet_time"]
    one_month_later = first_day + datetime.timedelta(days=+30)
    first_month_bets = player_bets[player_bets["bet_time"] <= one_month_later]
    result = first_month_bets["bet_count"].mean()
    return result

In [ ]:
#export
def frequency_raw(player_bets):
    "Number of active betting days in first month, if data set contains daily aggregate data."
    
    first_day = player_bets.iloc[0]["bet_time"]
    one_month_later = first_day + datetime.timedelta(days=+30)
    first_month_bets = player_bets[player_bets["bet_time"] <= one_month_later]

    return len(first_month_bets)

### Cost Domain

The cost domain contains all behavioural measures which can be computed using additional values from the ***bet_size*** column. As such, they describe *spending* behaviours as opposed to *engagement* behaviours as above.

In [ ]:
#export
def total_wagered(player_bets):
    "The total amount wagered (sum of bet sizes)."
    check_measure_data(player_bets, ["bet_size"])
    return player_bets["bet_size"].sum()

In [ ]:
#export
def mean_bet_size(player_bets, daily=False):
    "The average (mean) size of bets."
    if daily:
        check_measure_data(player_bets, ["bet_size"])
        mean_bet_size = player_bets["bet_size"].sum() / number_of_bets(player_bets, daily=True)
        return mean_bet_size
        
    check_measure_data(player_bets, ["bet_size"])
    mean_bet_size = player_bets["bet_size"].sum() / len(player_bets)
    return mean_bet_size

In [ ]:
#export
def variability(player_bets, daily=False):
    "Standard deviation of stake size in first month, if data set contains daily aggregate data."
    
    if daily:
        first_day = player_bets.iloc[0]["bet_time"]
        one_month_later = first_day + datetime.timedelta(days=+30)
        first_month_bets = player_bets[player_bets["bet_time"] <= one_month_later]

        return first_month_bets["bet_size"].std()

In [ ]:
#export
from sklearn.linear_model import LinearRegression
import numpy as np
def trajectory(player_bets, daily=False, plot=False):
    "Gradient of a linear regression fitted to the sequence of daily aggredated bet sizes."
    
    if daily:
        first_day = player_bets.iloc[0]["bet_time"]
        one_month_later = first_day + datetime.timedelta(days=+30)
        first_month_bets = player_bets[player_bets["bet_time"] <= one_month_later]

        # first_month_bets['mean_bet_size'] = first_month_bets['bet_size'] / first_month_bets['bet_count']

        x = np.array(range(len(first_month_bets))).reshape((-1, 1)) + 1
        y = first_month_bets["bet_size"].values

        model = LinearRegression().fit(x, y)
        r_sq = model.score(x, y)

        model_x = x
        model_y = model.coef_ * x + model.intercept_

        if plot:
            plt.figure()
            plt.scatter(x, y)
            plt.plot(model_x, model_y, color="r")
            plt.show()

        trajectory = model.coef_[0]
        return model.coef_[0]

In [ ]:
#export
def std_bet_sizes(player_bets):
    "Standard deviation of bet sizes (generalised version of `variability`)."
    check_measures_data(player_bets, ['bet_size'])
    
    return player_bets['bet_size'].std()

### Loss Domain

The loss domain includes all behavioural measures which require the additional information of the size of the payout received as a result of each bet. This data should be held in the ***payout_size*** column if available.

In [ ]:
#export
def net_loss(player_bets):
    "The net amount lost (sum of bet sizes minus sum of payout sizes)."
    check_measure_data(player_bets, ["bet_size", "payout_size"])
    net_loss_value = player_bets["bet_size"].sum() - player_bets["payout_size"].sum()
    return net_loss_value

In [ ]:
#export
def percent_loss(player_bets):
    "The `net_loss` as a percentage of `total_wagered`."

    net_loss_value = net_loss(player_bets)
    total_wagered_value = total_wagered(player_bets)
    percent_loss_value = (net_loss_value / total_wagered_value) * 100
    return percent_loss_value

### Other Domain

All measures not in the root, bet, or loss domains, require some additional information such as the house edge for each bet, the game being played, etc. These are currently broadly grouped into the 'other' domain' but as the library (and academic)'s capabilities grow, this domain will likely be split further.

In [ ]:
#export
def theoretical_loss(player_bets):
    pass

## Study Collections

The measures module also contains some convenience methods which accept full data sets of players bets and compute the collection of behavioural measures used in a given study.

In [ ]:
#export
from tqdm import tqdm
import datetime, pandas as pd
def calculate_labrie_measures(all_player_bets, savedir="", filename="gamba_labrie_measures.csv", loud=False, daily=True,):
    "Calculates the set of measures described in LaBrie et al's work in 2008 on casino gamblers. These measures include the durations, frequencies, number of bets, bets per day, value per bet (eth), total amount wagered, net loss, and percent loss for each player. As this method sits in the studies module, it accepts a list of dataframes representing each player's bets as input. By default, this method saves the resulting dataframe of each player's measures to 'gamba_labrie_measures.csv'. Be advised: this method can take some time for large numbers of players, the 'loud' parameter can be set to True to print out updates every 200 players."

    pd.options.mode.chained_assignment = None # supress setwithcopy warnings
    
    player_id = []
    all_duration = []
    all_frequency = []
    all_number_of_bets = []
    all_mean_bets_per_day = []
    all_mean_bet_size = []
    all_total_wagered = []
    all_net_loss = []
    all_percent_loss = []

    unique_players = list(set(all_player_bets["player_id"]))

    for i in tqdm(range(len(unique_players))):
        player_bets = all_player_bets[all_player_bets["player_id"] == unique_players[i]]
        player_id.append(player_bets.iloc[0]["player_id"])
        all_duration.append(duration(player_bets))
        all_frequency.append(frequency(player_bets))
        if daily:
            all_number_of_bets.append(number_of_bets(player_bets, daily=True))
            all_mean_bets_per_day.append(mean_bets_per_day(player_bets, daily=True))
            all_mean_bet_size.append(mean_bet_size(player_bets, daily=True))
        else:
            all_number_of_bets.append(number_of_bets(player_bets))
            all_mean_bets_per_day.append(mean_bets_per_day(player_bets))
            all_mean_bet_size.append(mean_bet_size(player_bets))
        all_total_wagered.append(total_wagered(player_bets))
        all_net_loss.append(net_loss(player_bets))
        all_percent_loss.append(percent_loss(player_bets))

    labrie_dict = {
        "player_id": player_id,
        "duration": all_duration,
        "frequency": all_frequency,
        "num_bets": all_number_of_bets,
        "mean_bets_per_day": all_mean_bets_per_day,
        "mean_bet_size": all_mean_bet_size,
        "total_wagered": all_total_wagered,
        "net_loss": all_net_loss,
        "percent_loss": all_percent_loss,
    }

    labrie_measures = pd.DataFrame.from_dict(labrie_dict)
    labrie_measures.to_csv(savedir + filename, index=False)

    if loud:
        print("LaBrie measures saved")

    return labrie_measures

In [ ]:
#export
def calculate_braverman_measures(all_player_bets, savedir="", loud=False):
    "Calculates the set of measures described in Braverman and Shaffer's work in 2010 on high risk internet gamblers. These measures include the frequency, intensity, variability, and trajectories of each player. As this method sits in the studies module, it accepts a list of dataframes representing each player's bets as input. By default, this method saves the resulting dataframe of each player's measures to 'gamba_braverman_measures.csv'."

    pd.options.mode.chained_assignment = None # supress setwithcopy warnings
    
    player_id = []

    intensities = []
    variabilities = []
    frequencies = []
    trajectories = []

    sum_of_stakes = []
    total_num_bets = []
    mean_bet_sizes = []
    durations = []
    net_losses = []

    unique_players = list(set(all_player_bets["player_id"]))

    for i in tqdm(range(len(unique_players))):
        player_bets = all_player_bets[all_player_bets["player_id"] == unique_players[i]]

        player_id.append(player_bets.iloc[0]["player_id"])
        intensities.append(intensity(player_bets))
        frequencies.append(frequency(player_bets))
        variabilities.append(variability(player_bets, daily=True))
        trajectories.append(trajectory(player_bets, daily=True))

        sum_of_stakes.append(player_bets["bet_size"].sum())
        total_num_bets.append(player_bets["bet_count"].sum())
        mean_bet_sizes.append(
            player_bets["bet_size"].sum() / player_bets["bet_count"].sum()
        )
        durations.append(duration(player_bets))
        net_losses.append(net_loss(player_bets))

    braverman_dict = {
        "player_id": player_id,
        "intensity": intensities,
        "frequency": frequencies,
        "variability": variabilities,
        "trajectory": trajectories,
        "sum_of_stakes": sum_of_stakes,
        "total_num_bets": total_num_bets,
        "mean_bet_size": mean_bet_sizes,
        "duration": durations,
        "net_loss": net_losses,
    }

    braverman_measures = pd.DataFrame.from_dict(braverman_dict)
    braverman_measures.to_csv(savedir + "gamba_braverman_measures.csv", index=False)

    return braverman_measures

## Utility Functions

This module also contains a utility function for standardising each of the measures in a measures table.

In [2]:
#export
def available_measures():
    "Returns a list of valid measures that the gamba library can compute. This list can be used in whole or in part by the `create_measures_table` method to compute these measures across one or more players bets."
    return ['duration','frequency','number_of_bets','mean_bets_per_day','intensity','frequency_raw','total_wagered','mean_bet_size','variability','trajectory','std_bet_sizes','net_loss','percent_loss']

In [ ]:
#export
import scipy.stats
def standardise_measures_table(measures_table):
    "Standardises all measures columns in a measures table by applying the scipy.stats.zscore function to each column. This is useful for column-wise comparisons and some clustering methods, but use with caution!"

    colnames = list(measures_table.columns)[1:]

    standardised_table = pd.DataFrame()
    standardised_table["player_id"] = measures_table["player_id"].values
    for col in colnames:
        standardised_table[col] = scipy.stats.zscore(measures_table[col].values)

    return standardised_table

In [ ]:
#export
def split_measures_table(measures_table, frac=0.7, loud=False):
	"Splits a measures table into two randomly selected groups. This is useful for machine learning methods where a train-test split is needed, and uses the Pandas library's sample method."
    
	measures_table.drop(['player_id'], axis=1)
	train_table = measures_table.sample(frac=frac) 
	test_table = measures_table.drop(train_table.index)

	if loud:
		print('train:test\n', len(train_table),':',len(test_table), 'ready')
	
	return train_table, test_table

In [1]:
#export
def check_measure_data(player_bets, required_columns):
    "Compares the columns found in a dataframe of player bets to a supplied list of column names. If any of the required_column names are not found, an exception is raised reporting the error."
    
    for column in required_columns:
        if column not in player_bets.columns:
            exception_string = (
                "Column '" + column + "' missing from provided dataframe."
            )
            raise Exception(exception_string)

## References

{% bibliography --cited %}